<a href="https://colab.research.google.com/github/fahime99/image_crawling/blob/main/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#These codes install the required packages
!pip install icrawler
!pip install easyocr
!pip install pyspellchecker
!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install deep-translator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.2/608.2 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 18.2 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.7.0.72
    Uninstalling opencv-python-headless-4.7.0.72:
      Successfully uninstalled opencv-python-headless-4.7.0.72
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-

In [2]:
#import the libraries
import datetime
import sys
import os,shutil
import re
from icrawler.builtin import google, GoogleImageCrawler
from spellchecker import SpellChecker
from fuzzywuzzy import fuzz
from typing import Literal
from string import ascii_letters
import textwrap
from PIL import ImageFont
from PIL import Image
from PIL import ImageDraw
from deep_translator import GoogleTranslator
import easyocr

In [3]:
def get_crawled_image_names(parent_path=None):
  list_ = os.listdir(parent_path)
  ext = ['jpg','bmp','jpeg']

  filtered = [x for x in list_ if any(y in x for y in ext)]
  
  return filtered

In [39]:
def generate_html_table(processed_data=None,directory_name=None):

    print("\nGenerateing HTML table started ...")

    if processed_data == None:
         print("Stoped: No processed data found!")
         return
    
    count = len(processed_data)
    str_rows=""
    row_header = "<tr>\n"\
                 "\t<td align='center'><h1>Quote</h1></td>\n"\
                 "\t<td align='center'><h1>Background</h1></td>\n"\
                 "\t<td align='center'><h1>Last File</h1></td>\n"\
                 "</tr>"

    for i in range(count):

          source = processed_data[i]["source_file"]

          wallpaper=processed_data[i]["background"]

          generated=processed_data[i]["generated"]
          row_str ="<tr>\n"\
                   "\t<td><img src='."+source+"' width='300' height='200'/></td>\n"\
                   "\t<td><img src='."+wallpaper+"' width='300' height='200'/></td>\n"\
                   "\t<td><img src='."+generated+"' width='300' height='200'/></td>\n"\
                   "</tr>"
          
          str_rows = str_rows + row_str
    
    print("Collected " +str(count)+" images ...")

    table_str = "<table align='center' border='1'>\n" + row_header + str_rows + "\n</table>"
    
    html_text = table_str
    try:
      readme = open(directory_name+"/README.MD", 'w')
      readme.write(html_text)
      readme.close() 

      print("HTML table saved in:", "/README.MD")
    except Exception as err:
      print(f"Unexpected {err=}, {type(err)=}")

    

    print()

In [5]:
google_crawler = GoogleImageCrawler()
google_crawler.crawl(keyword = "Anne shirley quote", offset = 0, max_num = 20, min_size = None, max_size = None)

ERROR:downloader:Response status code 429, file https://ciarraistudios.files.wordpress.com/2014/02/white-way-of-delight.jpg


In [6]:
if os.path.exists("/content/quotes"):
  shutil.rmtree("/content/quotes")
os.rename("/content/images", "/content/quotes")

In [26]:
google_crawler = GoogleImageCrawler()
google_crawler.crawl(keyword = "flower Background", offset = 0, max_num = 25, min_size = None, max_size = None)

ERROR:downloader:Response status code 400, file https://media.istockphoto.com/id/1294181713/vector/retro-delicate-wedding-card-with-pink-watercolor-texture-and-flowers.jpg
ERROR:downloader:Response status code 403, file https://img.rawpixel.com/private/static/images/website/2022-05/px1605560-image-kwvxq3bb.jpg
ERROR:downloader:Exception caught when downloading file https://png.png, error: HTTPSConnectionPool(host='png.png', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f16bb076190>: Failed to establish a new connection: [Errno -2] Name or service not known')), remaining retry times: 2
ERROR:downloader:Exception caught when downloading file https://png.png, error: HTTPSConnectionPool(host='png.png', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f16bb06ec40>: Failed to establish a new connection: [Errno -2] Name or service not known')

In [27]:
if os.path.exists("/content/images"):
  os.rename("/content/images","/content/backgrounds")

In [28]:
translator = GoogleTranslator(source='auto', target='fa')
font = ImageFont.truetype(font = '/content/Lalezar-Regular.ttf',size= 50)
processed_images = []

In [29]:
anne_img_names = get_crawled_image_names("/content/quotes")
background_img_names = get_crawled_image_names("/content/backgrounds")

In [30]:
len(background_img_names)

23

In [35]:
for i in range(20):
  anne_img_name = "/content/quotes/" + anne_img_names[i]
  background_img_name = "/content/backgrounds/" + background_img_names[i]
  generated_img_name = "/content/final/"+ str(i + 1)+".jpg"

  easyocr_reader = easyocr.Reader(["en"], gpu=False)
  list_of_text = easyocr_reader.readtext(anne_img_name, detail = 0)

  text = str('')
  for item in list_of_text:
    text = text + ' '+ item

  compiled = re.compile(re.escape('anne'), re.IGNORECASE)
  text = compiled.sub('', text)
  compiled = re.compile(re.escape('shirley'), re.IGNORECASE)
  text = compiled.sub('', text)
  text = text.replace('\n',' ')
  text = re.sub('\s+', ' ', text)

  words = text.split()
  final_text=""
  spell = SpellChecker()
  for word in words:
    misspelled = spell.unknown([word])
    if len(misspelled)==1:
      correct = spell.correction(word)
      if correct == None:
        final_text = final_text+ " "+ word
      else:
        final_text = final_text+" "+ spell.correction(word)
    else:
      if final_text =="":
        final_text = word
      else:
        final_text = final_text+ " "+ word

  best_file=""
  best_match = 0
  list_ = os.listdir("/content/anne_books")
  for f in list_:
    path_ = "/content/anne_books" +"/"+ str(f) 
    file = open(path_,"r")
    content = file.read()
    ratio = fuzz.token_set_ratio(final_text, content)
    if best_match < ratio:
      best_match = ratio
      best_file = f

  if best_match < 0.85 :
    continue

  translated = translator.translate(text=final_text)

  final_img = Image.open(background_img_name)
  draw = ImageDraw.Draw(final_img)
  width  = final_img.width
  height = final_img.height
  img_center = (width/2,height/2)
      
  # translated is the text you want to add to the image
  translated = translator.translate(text=final_text)

  # Open the image file
  final_img = Image.open(background_img_name)

  # Get a drawing context
  draw = ImageDraw.Draw(final_img)

  # Load the font
  font = ImageFont.truetype(font = '/content/Lalezar-Regular.ttf',size= 50)

  # Split the text into words
  words = translated.split()

  # Initialize variables
  x = 50
  y = 50
  line_width = 0
  line_text = ""

  # Loop through all the words
  for word in words:

     # Add the word to the line_text
     line_text += word + " "
    
     # Get the size of the line_text
     w, h = draw.textsize(line_text, font=font)
    
     # If the line width exceeds the image width or we have reached 6 words
     if w > final_img.width - 100 or len(line_text.split()) == 6:
        
        # Draw the line_text on the image
        draw.text((x, y), line_text, font=font, fill=(255, 255, 255))
        
        # Reset line_text and update the y coordinate
        line_text = ""
        y += h + 10
        
     # Update the line_width with the current word's width
     line_width += w

  # Save the modified image
  final_img.save(generated_img_name)

  anne_img_name = anne_img_name.replace("/content/drive/MyDrive/Colab Notebooks/Image_Crawling.ipynb","")
  background_img_name = background_img_name.replace("/content/drive/MyDrive/Colab Notebooks/Image_Crawling.ipynb","")
  generated_img_name = generated_img_name.replace("/content/drive/MyDrive/Colab Notebooks/Image_Crawling.ipynb","")

  processed_images.append({"source_file":anne_img_name, "background":background_img_name ,"generated":generated_img_name})

  print("Generated image(text match:"+str(best_match)+"%) #",generated_img_name)
  

Generated image(text match:100%) # /content/final/1.jpg


Generated image(text match:97%) # /content/final/2.jpg


Generated image(text match:77%) # /content/final/3.jpg


Generated image(text match:98%) # /content/final/4.jpg


Generated image(text match:82%) # /content/final/5.jpg


...

Generated image(text match:82%) # /content/final/15.jpg


Generated image(text match:73%) # /content/final/16.jpg


Generated image(text match:89%) # /content/final/17.jpg


Generated image(text match:65%) # /content/final/18.jpg


Generated image(text match:79%) # /content/final/19.jpg
Generated image(text match:95%) # /content/final/20.jpg


In [40]:
generate_html_table(processed_data = processed_images, directory_name = "/content/drive/MyDrive/Colab Notebooks/Image_Crawling.ipynb")


Generateing HTML table started ...
Collected 60 images ...
Unexpected err=FileNotFoundError(2, 'No such file or directory'), type(err)=<class 'FileNotFoundError'>



In [41]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
